In [1]:
import numpy as np
import matplotlib.pyplot as plt

### 기울기 폭발 알아보기

In [4]:
# RNN Matmul 노드의 역전파

N = 2   # 미니배치 크기
H = 3   # 은닉 상태 벡터의 차원 수
T = 20  # 시계열 데이터의 길이

dh = np.ones((N, H))
np.random.seed(3)       # 재현할 수 있는 난수의 시드 고정
Wh = np.random.randn(H, H)
print(dh)
print(Wh)

norm_list = []
for t in range(T):
    dh = np.matmul(dh, Wh.T)
    norm = np.sqrt(np.sum(dh**2)) / N
    norm_list.append(norm)
    print(norm)

[[1. 1. 1.]
 [1. 1. 1.]]
[[ 1.78862847  0.43650985  0.09649747]
 [-1.8634927  -0.2773882  -0.35475898]
 [-0.08274148 -0.62700068 -0.04381817]]
2.4684068094579303
3.3357049741610365
4.783279375373182
6.279587332087612
8.080776465019053
10.251163032292936
12.936063506609896
16.276861327786712
20.45482961834598
25.688972842084684
32.25315718048336
40.48895641683869
50.8244073070191
63.79612654485427
80.07737014308985
100.5129892205125
126.16331847536823
158.35920648258823
198.7710796761195
249.495615421267


### 기울기 소실

In [5]:
N = 2   # 미니배치 크기
H = 3   # 은닉 상태 벡터의 차원 수
T = 20  # 시계열 데이터의 길이

dh = np.ones((N, H))
np.random.seed(3)       # 재현할 수 있는 난수의 시드 고정
Wh = np.random.randn(H, H) * 0.5    # 초기값 변경
print(dh)
print(Wh)

norm_list = []
for t in range(T):
    dh = np.matmul(dh, Wh.T)
    norm = np.sqrt(np.sum(dh**2)) / N
    norm_list.append(norm)
    print(norm)

[[1. 1. 1.]
 [1. 1. 1.]]
[[ 0.89431424  0.21825493  0.04824873]
 [-0.93174635 -0.1386941  -0.17737949]
 [-0.04137074 -0.31350034 -0.02190908]]
1.2342034047289652
0.8339262435402591
0.5979099219216477
0.39247420825547574
0.2525242645318454
0.16017442237957713
0.10106299614538981
0.06358148956166684
0.03995083909833199
0.025086887541098325
0.015748611904532892
0.009884999125204758
0.006204151282595105
0.003893806551809953
0.002443767399386287
0.0015337065005571365
0.0009625497320203265
0.0006040924319556741
0.00037912574706291106
0.00023793756048323344


## 기울기 클리핑
### 기울기 폭발을 막는 전통적인 기법

In [8]:
dW1 = np.random.rand(3, 3) * 10
dW2 = np.random.rand(3, 3) * 10
grads = [dW1, dW2]
max_norm = 5.0

def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    if rate < 1:
        for grad in grads:
            grad *= rate